## Pass Order to API

**Note:** Make sure you set your gateway id

1. Execute post script to generate the 5 new records, wait about 5 seconds.

2. Execute get script to retrieve a record

3. Repeat the steps to see the *account_total* increasing 

Records are loaded into 2 DynamoDB tables. 

**dynamodb_stream_cqrs_order** - holds the details for the orders

**dynamodb_stream_cqrs_account_summary** - holds the account summary


In [ ]:
#Post script

import json, boto3, requests, datetime
from decimal import Decimal

#Set gateway id
gwid = ''
    
url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/order')
    
for i in range(5):
    y = i+1
    x = "id_" + str(i) + str(datetime.datetime.now())
    orderid = str(x)
    accountid = 'p' + str(y)
    vendorid = 'v' + str(y)
    orderdate = str(x)
    coffeetype = 'Short Black'
    coffeesize = 'Small'
    unitprice = '4.50'
    quantity = y
        
    response = requests.post(url,json={'order':{
                'orderid': orderid,
                'accountid': accountid,
                'vendorid': vendorid,
                'orderdate':orderdate,
                'details':{
                    'coffeetype': coffeetype,
                    'coffeesize': coffeesize,
                    'unitprice': unitprice,
                    'quantity': quantity
                }
            }
        })
    print(response)

Query the account summary table

In [ ]:
#Get Script

url = (f'https://{gwid}.execute-api.ap-southeast-2.amazonaws.com/prod/summary')

response_get = requests.get(url, params={'accountid':'p1'})

print(response_get.json())